In [1]:
import requests
import pandas as pd
import tqdm
from dotenv import load_dotenv
import os

In [2]:
load_dotenv(dotenv_path='credentials.env')

api_key = os.getenv("API_KEY")
username = os.getenv("USER")

In [14]:
#url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/crear_usuario/?mail={username}'

In [15]:
#res = requests.post(url)
#res.json()

In [3]:
creditos_hist = pd.read_parquet('./input/creditos_hist.parquet')

In [4]:
url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/consulta_constancia_masiva/?usuario={username}&api_key={api_key}'

In [5]:
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

In [6]:
def resultados_constancias(respuesta):
    datos = {}
    res = respuesta.get('resultados', {})
    
    datos_generales = res.get('datosGenerales') if res.get('datosGenerales') else None
    datos_regimen = res.get('datosRegimenGeneral') if res.get('datosRegimenGeneral') else None
    error_constancia = res.get('errorConstancia') if res.get('errorConstancia') else None
    error_regimen = res.get('errorRegimenGeneral') if res.get('errorRegimenGeneral') else None
    
    # Datos Generales
    if datos_generales:
        datos['cuit'] = datos_generales.get('idPersona')
    elif error_constancia:
        datos['cuit'] = error_constancia.get('idPersona')
    else:
        datos['cuit'] = None
        
    datos['mes_cierre'] = datos_generales.get('mesCierre') if datos_generales else None
    datos['razon_social'] = datos_generales.get('razonSocial') if datos_generales else None
    datos['contrato_social'] = datos_generales.get('fechaContratoSocial') if datos_generales else None
    datos['tipo_persona'] = datos_generales.get('tipoPersona') if datos_generales else None
    
    # Domicilio Fiscal
    domicilio_fiscal = datos_generales.get('domicilioFiscal') if datos_generales else None
    datos['codigo_postal'] = domicilio_fiscal.get('codPostal') if domicilio_fiscal else None
    datos['provincia'] = domicilio_fiscal.get('descripcionProvincia') if domicilio_fiscal else None
    datos['direccion'] = domicilio_fiscal.get('direccion') if domicilio_fiscal else None
    datos['localidad'] = domicilio_fiscal.get('localidad') if domicilio_fiscal else None
    
    # Datos Régimen
    datos_actividad = datos_regimen.get('actividad', []) if datos_regimen else None
    impuestos = datos_regimen.get('impuesto', []) if datos_regimen else None
    regimenes = datos_regimen.get('regimen', []) if datos_regimen else None
    
    # Error en la constancia
    datos['errorConstancia'] = error_constancia.get('error') if error_constancia else None
    
    # Error en Régimen General
    datos['errorRegimenGeneral'] = error_regimen.get('error') if error_regimen else None
    
    # Actividad
    if datos_actividad:
        datos['actividades'] = '; '.join([
            f"{a.get('descripcionActividad')} ({a.get('idActividad')})" 
            for a in datos_actividad
        ])
        for actividad in datos_actividad:
            id = actividad.get('idActividad')
            datos[f'actividad_{id}'] = True
    
    # Impuestos
    if impuestos:
        datos['impuestos'] = '; '.join([
			f"{i.get('descripcionImpuesto')} ({i.get('idImpuesto')})"
			for i in impuestos
		])
        for impuesto in impuestos:
            id = impuesto.get('idImpuesto')
            datos[f'impuestos_{id}'] = True
    
    # Regímenes
    if regimenes:
        datos['regimenes'] = '; '.join([
            f"{r.get('descripcionRegimen')} ({r.get('idRegimen')})"
			for r in regimenes
		])
        for regimen in regimenes:
            id = regimen.get('idRegimen')
            datos[f'regimen_{id}'] = True
            
    return datos

In [16]:
def resultados_constancias(respuesta):
    datos = {}
    res = respuesta.get('resultados', {})
    
    # Extracción de secciones principales usando get() con default
    datos_generales = res.get('datosGenerales', {})
    datos_regimen = res.get('datosRegimenGeneral', {})
    error_constancia = res.get('errorConstancia', {})
    error_regimen = res.get('errorRegimenGeneral', {})
    
    # 1. Manejo de CUIT (corregido error de sintaxis y lógica)
    datos['cuit'] = (
        datos_generales.get('idPersona') or 
        error_constancia.get('idPersona') or 
        None
    )
    
    # 2. Datos generales (versión simplificada)
    general_fields = {
        'mes_cierre': 'mesCierre',
        'razon_social': 'razonSocial',
        'contrato_social': 'fechaContratoSocial',
        'tipo_persona': 'tipoPersona'
    }
    for field, key in general_fields.items():
        datos[field] = datos_generales.get(key)
    
    # 3. Domicilio fiscal (usando dict.get() encadenado)
    domicilio = datos_generales.get('domicilioFiscal', {})
    domicilio_fields = {
        'codigo_postal': 'codPostal',
        'provincia': 'descripcionProvincia',
        'direccion': 'direccion',
        'localidad': 'localidad'
    }
    for field, key in domicilio_fields.items():
        datos[field] = domicilio.get(key)
    
    # 4. Manejo de errores (convertimos listas a string)
    datos['error_constancia'] = '; '.join(error_constancia.get('error', [])) or None
    datos['error_regimen'] = '; '.join(error_regimen.get('error', [])) or None
    
    # 5. Procesamiento de régimen (evitamos None usando get con default)
    def procesar_seccion(seccion, nombre, desc_key, id_key):
        items = datos_regimen.get(seccion, [])
        if items:
            datos[f'{nombre}_listado'] = '; '.join(
                f"{i.get(desc_key, '')} ({i.get(id_key, '')})" 
                for i in items
            )
            for item in items:
                if (item_id := item.get(id_key)) is not None:
                    datos[f'{nombre}_{item_id}'] = True
    
    procesar_seccion('actividad', 'actividad', 'descripcionActividad', 'idActividad')
    procesar_seccion('impuesto', 'impuesto', 'descripcionImpuesto', 'idImpuesto')
    procesar_seccion('regimen', 'regimen', 'descripcionRegimen', 'idRegimen')
    
    return datos

In [10]:
identificaciones = creditos_hist['identificacion'].unique()
tramos = [identificaciones[i:i + 250] for i in range(0, len(identificaciones), 250)]

In [ ]:
todos_los_datos = []
session = requests.Session()

for tramo in tqdm.tqdm(tramos):
    try:
        payload = {
        	"cuits": [str(cuit) for cuit in tramo]
        }
        response = session.post(url, json=payload, headers=headers)
        response.raise_for_status()
        todos_los_datos.extend(response.json())
    except Exception as e:
        print(e)

In [ ]:
todos_los_datos = []
session = requests.Session()
for cuit in tqdm.tqdm(creditos_hist['identificacion'].unique()[:10]):
    try:
        url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/consulta_constancia/?cuit={cuit}&usuario={username}&api_key={api_key}'
        response = session.get(url, headers= headers)
        response.raise_for_status()
        todos_los_datos.append(response.json())
    except Exception as e:
        print(e) 

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
prueba = []
for respuesta in todos_los_datos:
    datos = resultados_constancias(respuesta)
    prueba.append(datos)

In [12]:
datos = pd.DataFrame(todos_los_datos)

In [16]:
datos.to_parquet('./input/constancia_inscripcion.parquet', compression= 'gzip')

In [18]:
faltan = creditos_hist.loc[~creditos_hist['identificacion'].isin(datos['identificacion'].unique())]['identificacion'].unique()

In [15]:
session = requests.Session()
prueba = []
for cuit in faltan[:10]:
    try:
        url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/consulta_constancia/?cuit={cuit}&usuario={username}&api_key={api_key}'
        response = session.get(url, headers= headers)
        response.raise_for_status()
        prueba.append(response.json())
    except Exception as e:
        print(e) 

In [2]:
arca = pd.read_parquet('./input/constancia_inscripcion.parquet')